In [ ]:
!ollama pull llama3.2

In [ ]:
import os
from dotenv import load_dotenv
from openai import OpenAI
from datetime import datetime
import gradio as gr

In [ ]:
load_dotenv()

openai_api_key = os.getenv("OPENAI_API_KEY")
anthropic_api_key = os.getenv("ANTHROPIC_API_KEY")
google_api_key = os.getenv("GOOGLE_API_KEY")
if openai_api_key:
    print("OpenAI API key loaded successfully.")
else:
    print("Error: OpenAI API key not found. Please set it in the .env file.")
if anthropic_api_key:
    print("Anthropic API key loaded successfully.")
else:
    print("Error: Anthropic API key not found. Please set it in the .env file.")
if google_api_key:
    print("Gemini API key loaded successfully.")
else:
    print("Error: Gemini API key not found. Please set it in the .env file.")

In [ ]:
anthropic_url = "https://api.anthropic.com/v1/"
gemini_url = "https://generativelanguage.googleapis.com/v1beta/openai/"
ollama_url = "http://localhost:11434/v1"

openai = OpenAI()
anthropic = OpenAI(base_url=anthropic_url, api_key=anthropic_api_key)
gemini = OpenAI(base_url=gemini_url, api_key=google_api_key)
ollama = OpenAI(base_url=ollama_url, api_key="ollama")

In [ ]:
OPENAI_MODEL = "gpt-4.1-mini"
ANTHROPIC_MODEL = "claude-haiku-4-5"
GEMINI_MODEL = "gemini-2.5-flash-lite"
OLLAMA_MODEL = "llama3.2"


In [ ]:
system_prompt = """
You are a helpful technical tutor who answers questions regarding software development, programming, and computer science.
"""

In [ ]:
def get_current_datetime():
    return f"Current date and time is: {datetime.now().strftime("%Y-%m-%d %H:%M")}"

In [ ]:
datetime_function = {
    "name": "get_current_datetime",
    "description": "Get the current date and time.",
    "parameters": {
        "type": "object",
        "properties": {},
        "required": [],
        "additionalProperties": False,
    },
}

In [ ]:
tools = [{"type": "function", "function": datetime_function}]

In [ ]:
def handle_tool_calls(message):
    responses = []
    for tool_call in message.tool_calls:
        if tool_call.function.name == "get_current_datetime":
            print(f"Handling tool call: {tool_call.function.name}")
            response = get_current_datetime()
            responses.append({"role": "tool", "content": response, "tool_call_id": tool_call.id})
    return responses

In [ ]:
def stream_gpt(messages, tools, stream):
    return openai.chat.completions.create(
    model=OPENAI_MODEL,
    messages=messages,
    tools=tools,
    stream=stream,
    )

In [ ]:
def stream_claude(messages, tools, stream):
    return anthropic.chat.completions.create(
        model=ANTHROPIC_MODEL,
        messages=messages,
        tools=tools,
        stream=stream,
    )

In [ ]:
def stream_gemini(messages, tools, stream):
    return gemini.chat.completions.create(
        model=GEMINI_MODEL,
        messages=messages,
        tools=tools,
        stream=stream,
    )

In [ ]:
def stream_ollama(messages, stream):
    return ollama.chat.completions.create(
        model=OLLAMA_MODEL,
        messages=messages,
        stream=stream,
    )

No tools for llama3.2, I removed it because I was getting blank replies sometimes.

In [ ]:
def handle_call(model, messages, tools, stream=False):
    if model == OPENAI_MODEL:
        return stream_gpt(messages, tools, stream)
    elif model == ANTHROPIC_MODEL:
        return stream_claude(messages, tools, stream)
    elif model == GEMINI_MODEL:
        return stream_gemini(messages, tools, stream)
    elif model == OLLAMA_MODEL:
        return stream_ollama(messages, stream)

In [ ]:
def echo(message, history, model):
    history = [{"role": h["role"], "content": h["content"]} for h in history]
    messages = (
        [{"role": "system", "content": system_prompt}]
        + history
        + [{"role": "user", "content": message}]
    )

    response = handle_call(model, messages, tools, stream=False)
    while response.choices[0].finish_reason == "tool_calls":
        message = response.choices[0].message
        responses = handle_tool_calls(message)
        messages.append(message)
        messages.extend(responses)
        response = handle_call(model, messages, tools, stream=False)

    stream = handle_call(model, messages, tools, stream=True)
    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ""
        yield response

In [ ]:
gr.ChatInterface(
    fn=echo,
    type="messages",
    title="Technical Tutor",
    additional_inputs=[
        gr.Dropdown(choices=[OPENAI_MODEL, ANTHROPIC_MODEL, GEMINI_MODEL, OLLAMA_MODEL])
    ],
).launch(share=True)